In [1]:
from __future__ import print_function
# import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [2]:
batch_size = 64

<h3>data</h3>

In [3]:
train_dataset = datasets.MNIST(root='./mnist_data',
                              train=True,
                              transform=transforms.ToTensor(),
                              download=True)

In [4]:
test_dataset = datasets.MNIST(root='./mnist_data',
                             train=False,
                             transform=transforms.ToTensor())

In [5]:
train_loader=torch.utils.data.DataLoader(dataset=train_dataset,
                                        batch_size = batch_size,
                                        shuffle=True)

In [6]:
test_loader=torch.utils.data.DataLoader(dataset=test_dataset,
                                       batch_size=batch_size,
                                       shuffle=False)

<h3>model</h3>

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1=nn.Conv2d(1,10,kernel_size=5) #2d에서 흑백(1차원)이니 input:1
        self.conv2=nn.Conv2d(10,20,kernel_size=5)
        self.mp=nn.MaxPool2d(2)
        self.fc=nn.Linear(320,10)
    def forward(self,x):
        in_size=x.size(0) #64
        x=F.relu(self.mp(self.conv1(x))) #28x28 -> 24x24 -> 12x12
        x=F.relu(self.mp(self.conv2(x))) #12x12 -> 8x8 -> 4x4
        x=x.view(in_size,-1) # 4x4 = 16 -> 16 x 20 = 320
        x=self.fc(x)
        return(F.log_softmax(x))

In [8]:
model = Net()

In [9]:
optimizer=optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

<h3>train & test</h3>

In [10]:
def train(epoch):
    model.train()
    for batch_idx, (data,target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        
        optimizer.zero_grad() #최적화된 모든 그라디언트를 지운다
        output=model(data)
        
        loss=F.nll_loss(output,target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 10 == 0:
            print('train epoch: {} [{}/{} ({:.0f}%)]\tloss: {:.6f}'.format(
                                epoch,
                                    batch_idx*len(data),
                                       len(train_loader.dataset),
                                           100.*batch_idx/len(train_loader),
                                                            loss.data[0]))

In [11]:
def test():
    model.eval()
    test_loss=0
    correct=0
    for data, target in test_loader:
        data, target = Variable(data, volatile = True), Variable(target)
        
        output = model(data)
        
        test_loss += F.nll_loss(output, target, size_average=False).data[0]
        pred = output.data.max(1,keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    
    test_loss /= len(test_loader.dataset)
    print('\ntest set: average loss: {:.4f}, accuracy: {}/{} ({:.0f}%)\n'.format(
                                     test_loss,
                                                       correct,
                                                          len(test_loader.dataset),
                                                              100.*correct/len(test_loader.dataset)))


In [12]:
for epoch in range(1,10):
    train(epoch)
    test()

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  


train epoch: 1 [0/60000 (0%)]	loss: 2.307797
train epoch: 1 [640/60000 (1%)]	loss: 2.307036
train epoch: 1 [1280/60000 (2%)]	loss: 2.289007
train epoch: 1 [1920/60000 (3%)]	loss: 2.287665
train epoch: 1 [2560/60000 (4%)]	loss: 2.278623
train epoch: 1 [3200/60000 (5%)]	loss: 2.250750
train epoch: 1 [3840/60000 (6%)]	loss: 2.245348
train epoch: 1 [4480/60000 (7%)]	loss: 2.261759
train epoch: 1 [5120/60000 (9%)]	loss: 2.225154
train epoch: 1 [5760/60000 (10%)]	loss: 2.195457
train epoch: 1 [6400/60000 (11%)]	loss: 2.169635
train epoch: 1 [7040/60000 (12%)]	loss: 2.101324
train epoch: 1 [7680/60000 (13%)]	loss: 1.982084
train epoch: 1 [8320/60000 (14%)]	loss: 1.776032
train epoch: 1 [8960/60000 (15%)]	loss: 1.544436
train epoch: 1 [9600/60000 (16%)]	loss: 1.297668
train epoch: 1 [10240/60000 (17%)]	loss: 1.059590
train epoch: 1 [10880/60000 (18%)]	loss: 1.062562
train epoch: 1 [11520/60000 (19%)]	loss: 0.862203
train epoch: 1 [12160/60000 (20%)]	loss: 0.750875
train epoch: 1 [12800/60000 (

train epoch: 2 [44800/60000 (75%)]	loss: 0.211723
train epoch: 2 [45440/60000 (76%)]	loss: 0.078749
train epoch: 2 [46080/60000 (77%)]	loss: 0.120718
train epoch: 2 [46720/60000 (78%)]	loss: 0.215147
train epoch: 2 [47360/60000 (79%)]	loss: 0.228667
train epoch: 2 [48000/60000 (80%)]	loss: 0.056172
train epoch: 2 [48640/60000 (81%)]	loss: 0.145993
train epoch: 2 [49280/60000 (82%)]	loss: 0.209078
train epoch: 2 [49920/60000 (83%)]	loss: 0.078628
train epoch: 2 [50560/60000 (84%)]	loss: 0.242736
train epoch: 2 [51200/60000 (85%)]	loss: 0.127122
train epoch: 2 [51840/60000 (86%)]	loss: 0.106824
train epoch: 2 [52480/60000 (87%)]	loss: 0.107521
train epoch: 2 [53120/60000 (88%)]	loss: 0.058171
train epoch: 2 [53760/60000 (90%)]	loss: 0.126132
train epoch: 2 [54400/60000 (91%)]	loss: 0.056640
train epoch: 2 [55040/60000 (92%)]	loss: 0.087921
train epoch: 2 [55680/60000 (93%)]	loss: 0.129010
train epoch: 2 [56320/60000 (94%)]	loss: 0.086966
train epoch: 2 [56960/60000 (95%)]	loss: 0.149596


train epoch: 4 [29440/60000 (49%)]	loss: 0.079366
train epoch: 4 [30080/60000 (50%)]	loss: 0.115552
train epoch: 4 [30720/60000 (51%)]	loss: 0.048581
train epoch: 4 [31360/60000 (52%)]	loss: 0.092489
train epoch: 4 [32000/60000 (53%)]	loss: 0.050238
train epoch: 4 [32640/60000 (54%)]	loss: 0.101953
train epoch: 4 [33280/60000 (55%)]	loss: 0.036110
train epoch: 4 [33920/60000 (57%)]	loss: 0.088831
train epoch: 4 [34560/60000 (58%)]	loss: 0.303578
train epoch: 4 [35200/60000 (59%)]	loss: 0.054032
train epoch: 4 [35840/60000 (60%)]	loss: 0.038279
train epoch: 4 [36480/60000 (61%)]	loss: 0.078716
train epoch: 4 [37120/60000 (62%)]	loss: 0.059581
train epoch: 4 [37760/60000 (63%)]	loss: 0.030148
train epoch: 4 [38400/60000 (64%)]	loss: 0.107145
train epoch: 4 [39040/60000 (65%)]	loss: 0.083834
train epoch: 4 [39680/60000 (66%)]	loss: 0.066119
train epoch: 4 [40320/60000 (67%)]	loss: 0.093504
train epoch: 4 [40960/60000 (68%)]	loss: 0.066929
train epoch: 4 [41600/60000 (69%)]	loss: 0.056917


train epoch: 6 [14080/60000 (23%)]	loss: 0.032526
train epoch: 6 [14720/60000 (25%)]	loss: 0.036269
train epoch: 6 [15360/60000 (26%)]	loss: 0.024374
train epoch: 6 [16000/60000 (27%)]	loss: 0.118194
train epoch: 6 [16640/60000 (28%)]	loss: 0.030952
train epoch: 6 [17280/60000 (29%)]	loss: 0.023852
train epoch: 6 [17920/60000 (30%)]	loss: 0.065904
train epoch: 6 [18560/60000 (31%)]	loss: 0.151793
train epoch: 6 [19200/60000 (32%)]	loss: 0.028399
train epoch: 6 [19840/60000 (33%)]	loss: 0.020977
train epoch: 6 [20480/60000 (34%)]	loss: 0.056256
train epoch: 6 [21120/60000 (35%)]	loss: 0.011726
train epoch: 6 [21760/60000 (36%)]	loss: 0.055726
train epoch: 6 [22400/60000 (37%)]	loss: 0.018450
train epoch: 6 [23040/60000 (38%)]	loss: 0.069458
train epoch: 6 [23680/60000 (39%)]	loss: 0.095170
train epoch: 6 [24320/60000 (41%)]	loss: 0.062233
train epoch: 6 [24960/60000 (42%)]	loss: 0.022853
train epoch: 6 [25600/60000 (43%)]	loss: 0.069253
train epoch: 6 [26240/60000 (44%)]	loss: 0.065115


train epoch: 7 [58880/60000 (98%)]	loss: 0.084417
train epoch: 7 [59520/60000 (99%)]	loss: 0.081694

test set: average loss: 0.0567, accuracy: 9824/10000 (98%)

train epoch: 8 [0/60000 (0%)]	loss: 0.011874
train epoch: 8 [640/60000 (1%)]	loss: 0.042873
train epoch: 8 [1280/60000 (2%)]	loss: 0.052841
train epoch: 8 [1920/60000 (3%)]	loss: 0.014555
train epoch: 8 [2560/60000 (4%)]	loss: 0.098954
train epoch: 8 [3200/60000 (5%)]	loss: 0.147345
train epoch: 8 [3840/60000 (6%)]	loss: 0.074430
train epoch: 8 [4480/60000 (7%)]	loss: 0.015701
train epoch: 8 [5120/60000 (9%)]	loss: 0.026396
train epoch: 8 [5760/60000 (10%)]	loss: 0.026033
train epoch: 8 [6400/60000 (11%)]	loss: 0.143371
train epoch: 8 [7040/60000 (12%)]	loss: 0.040465
train epoch: 8 [7680/60000 (13%)]	loss: 0.015093
train epoch: 8 [8320/60000 (14%)]	loss: 0.061424
train epoch: 8 [8960/60000 (15%)]	loss: 0.045837
train epoch: 8 [9600/60000 (16%)]	loss: 0.108430
train epoch: 8 [10240/60000 (17%)]	loss: 0.018065
train epoch: 8 [10

train epoch: 9 [43520/60000 (72%)]	loss: 0.011144
train epoch: 9 [44160/60000 (74%)]	loss: 0.016583
train epoch: 9 [44800/60000 (75%)]	loss: 0.079721
train epoch: 9 [45440/60000 (76%)]	loss: 0.164593
train epoch: 9 [46080/60000 (77%)]	loss: 0.094597
train epoch: 9 [46720/60000 (78%)]	loss: 0.007125
train epoch: 9 [47360/60000 (79%)]	loss: 0.019640
train epoch: 9 [48000/60000 (80%)]	loss: 0.039752
train epoch: 9 [48640/60000 (81%)]	loss: 0.083201
train epoch: 9 [49280/60000 (82%)]	loss: 0.076287
train epoch: 9 [49920/60000 (83%)]	loss: 0.032513
train epoch: 9 [50560/60000 (84%)]	loss: 0.074280
train epoch: 9 [51200/60000 (85%)]	loss: 0.011453
train epoch: 9 [51840/60000 (86%)]	loss: 0.054536
train epoch: 9 [52480/60000 (87%)]	loss: 0.044158
train epoch: 9 [53120/60000 (88%)]	loss: 0.082740
train epoch: 9 [53760/60000 (90%)]	loss: 0.027028
train epoch: 9 [54400/60000 (91%)]	loss: 0.092829
train epoch: 9 [55040/60000 (92%)]	loss: 0.030360
train epoch: 9 [55680/60000 (93%)]	loss: 0.012647


In [19]:
model

Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (mp): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
  (fc): Linear(in_features=320, out_features=10, bias=True)
)

In [21]:
model.forward()

<bound method Net.forward of Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (mp): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
  (fc): Linear(in_features=320, out_features=10, bias=True)
)>